# Taller 3 MLOps - Data Science Salaries 2023
## Fase 2: Preparación de Datos

### Universidad EIA
Transformación y preparación de datos para modelación

## 1. IMPORTAR LIBRERÍAS

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
import os

print("✓ Librerías importadas correctamente")

✓ Librerías importadas correctamente


## 2. CARGAR DATOS DEL NOTEBOOK 1

In [22]:
# Cargar datos crudos
df = pd.read_csv('../data/ds_salaries.csv')

print(f"Dataset cargado: {df.shape}")
print(f"\nPrimeras filas:")
print(df.head())
print(f"\nColumnas: {df.columns.tolist()}")

Dataset cargado: (3755, 11)

Primeras filas:
   work_year experience_level employment_type                 job_title  \
0       2023               SE              FT  Principal Data Scientist   
1       2023               MI              CT               ML Engineer   
2       2023               MI              CT               ML Engineer   
3       2023               SE              FT            Data Scientist   
4       2023               SE              FT            Data Scientist   

   salary salary_currency  salary_in_usd employee_residence  remote_ratio  \
0   80000             EUR          85847                 ES           100   
1   30000             USD          30000                 US           100   
2   25500             USD          25500                 US           100   
3  175000             USD         175000                 CA           100   
4  120000             USD         120000                 CA           100   

  company_location company_size  
0      

## 3. EXPLORAR VALORES FALTANTES

In [23]:
# Valores faltantes
print("Valores faltantes:")
print(df.isnull().sum())
print(f"\nSin valores faltantes: {df.isnull().sum().sum() == 0}")

Valores faltantes:
work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

Sin valores faltantes: True


## 4. IDENTIFICAR TIPOS DE VARIABLES

In [24]:
# Separar variables
numerical_features = ['work_year', 'remote_ratio']
categorical_features = ['experience_level', 'employment_type', 'job_title', 'company_size', 'company_location']
target = 'salary_in_usd'   # ✔️ Corrección clave

print(f"Variables numéricas: {numerical_features}")
print(f"Variables categóricas: {categorical_features}")
print(f"Variable objetivo: {target}")

# Separar X (features) y y (target)
X = df.drop(columns=[target])
y = df[target]

print(f"\nX shape: {X.shape}")
print(f"y shape: {y.shape}")


Variables numéricas: ['work_year', 'remote_ratio']
Variables categóricas: ['experience_level', 'employment_type', 'job_title', 'company_size', 'company_location']
Variable objetivo: salary_in_usd

X shape: (3755, 10)
y shape: (3755,)


## 5. DIVIDIR EN TRAIN, VALIDATION Y TEST

In [25]:
from sklearn.model_selection import train_test_split

# Dividir en train+val y test (80% - 20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Dividir train+val en train y val (75% - 25% del temp)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42
)

print(f"Train: {X_train.shape[0]} ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Val:   {X_val.shape[0]} ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"Test:  {X_test.shape[0]} ({X_test.shape[0]/len(X)*100:.1f}%)")

Train: 2253 (60.0%)
Val:   751 (20.0%)
Test:  751 (20.0%)


## 6. CREAR PIPELINE DE PREPROCESAMIENTO

In [26]:
# Pipeline de transformación
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ]
)

print("✓ Pipeline de preprocesamiento creado")

# Entrenar el pipeline con datos de train
X_train_processed = preprocessor.fit_transform(X_train)
print(f"\nDatos de train procesados: {X_train_processed.shape}")

# Transformar val y test
X_val_processed = preprocessor.transform(X_val)
X_test_processed = preprocessor.transform(X_test)

print(f"Datos de val procesados: {X_val_processed.shape}")
print(f"Datos de test procesados: {X_test_processed.shape}")

✓ Pipeline de preprocesamiento creado

Datos de train procesados: (2253, 157)
Datos de val procesados: (751, 157)
Datos de test procesados: (751, 157)


## 7. CONVERTIR A DATAFRAMES (OPCIONAL PARA CLARITY)

In [27]:
# Obtener nombres de features después del OneHotEncoder
feature_names = (
    numerical_features + 
    list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))
)

# Convertir a DataFrames
X_train_df = pd.DataFrame(X_train_processed, columns=feature_names).reset_index(drop=True)
X_val_df = pd.DataFrame(X_val_processed, columns=feature_names).reset_index(drop=True)
X_test_df = pd.DataFrame(X_test_processed, columns=feature_names).reset_index(drop=True)

y_train_reset = y_train.reset_index(drop=True)
y_val_reset = y_val.reset_index(drop=True)
y_test_reset = y_test.reset_index(drop=True)

print(f"Train X shape: {X_train_df.shape}")
print(f"Train y shape: {y_train_reset.shape}")

Train X shape: (2253, 157)
Train y shape: (2253,)


## 8. GUARDAR DATOS PROCESADOS

In [28]:
import os
import joblib

# Crear carpeta si no existe
os.makedirs('../data/processed', exist_ok=True)
os.makedirs('../models', exist_ok=True)

# Guardar datos procesados
X_train_df.to_csv('../data/processed/X_train.csv', index=False)
X_val_df.to_csv('../data/processed/X_val.csv', index=False)
X_test_df.to_csv('../data/processed/X_test.csv', index=False)

y_train_reset.to_csv('../data/processed/y_train.csv', index=False)
y_val_reset.to_csv('../data/processed/y_val.csv', index=False)
y_test_reset.to_csv('../data/processed/y_test.csv', index=False)

# Guardar el preprocessor
joblib.dump(preprocessor, '../models/preprocessor.pkl')

print("Datos procesados guardados en ../data/processed/")
print("Preprocessor guardado en ../models/preprocessor.pkl")


Datos procesados guardados en ../data/processed/
Preprocessor guardado en ../models/preprocessor.pkl


## 9. RESUMEN DE LA FASE 2

In [29]:
print("\n" + "="*70)
print("RESUMEN - FASE 2: PREPARACIÓN DE DATOS")
print("="*70)

print(f"""
✓ DATOS ORIGINALES:
  Filas: {len(df)}
  Columnas: {len(df.columns)}

✓ DIVISIÓN:
  Train: {len(X_train_df)} muestras
  Val:   {len(X_val_df)} muestras
  Test:  {len(X_test_df)} muestras

✓ TRANSFORMACIONES:
  Variables numéricas: {len(numerical_features)} (StandardScaler)
  Variables categóricas: {len(categorical_features)} (OneHotEncoder)
  Features finales: {len(feature_names)}

✓ ARCHIVOS GENERADOS:
  X_train.csv, X_val.csv, X_test.csv
  y_train.csv, y_val.csv, y_test.csv
  preprocessor.pkl

✓ PRÓXIMO PASO:
  Ejecutar Notebook 03: Modelación (Optuna + MLFlow)
""")

print("="*70)


RESUMEN - FASE 2: PREPARACIÓN DE DATOS

✓ DATOS ORIGINALES:
  Filas: 3755
  Columnas: 11

✓ DIVISIÓN:
  Train: 2253 muestras
  Val:   751 muestras
  Test:  751 muestras

✓ TRANSFORMACIONES:
  Variables numéricas: 2 (StandardScaler)
  Variables categóricas: 5 (OneHotEncoder)
  Features finales: 157

✓ ARCHIVOS GENERADOS:
  X_train.csv, X_val.csv, X_test.csv
  y_train.csv, y_val.csv, y_test.csv
  preprocessor.pkl

✓ PRÓXIMO PASO:
  Ejecutar Notebook 03: Modelación (Optuna + MLFlow)

